In [19]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch
import random

import nltk
nltk.download('punkt')
'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lusty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\nChatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.\n\nImagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.\n\nCreate a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.\n\nThe final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.\n\nExample\nA: Hello MedAssistant.\nB: Hello. How can I help you?\nA: I don't feel well.\nB: Do you have any symptoms?\nA: I have cough and nausea.\nB: Do you want to book an appointment?\nA: Yes, please, for tomorrow.\n\n\nInfo:\n- Feel free to change or arrange a new dataset of intents\n- Try experimenting and tunin

In [20]:
import numpy as np
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# STEP 0: find intents patterns

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
    tags.append(intent['tag'])
    for pattern in intent['patterns']:

        # Taking each word and tokenizing them:
        w = nltk.word_tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, intent['tag']))
# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns

print(patterns)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Good', 'evening'], 'greeting'), (['Good', 'night'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'), (['Which', 'hamburgers', 'do', 'you', 'have', '?'], 'food'), (['What', 'kinds', 'of', 'courses', 'are', 'there', '?'], 'food'), (['What', 'do', 'you', 'cook', '?'], 'food'), (['Can', 'I', 'order', 'a', 'coke', '?'], 'food'), (['I', "'m", 'hungry'], 'food'), (['We', "'re", 'starving'], 'food'), (['I', "'d", 'like', 'to', 'have', 'a', 'pizza'], 'food'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Can', 'I', 'pay', 'with', 'Apple', 'Pay', 'or'

In [21]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD
stop_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in stop_words]
all_words = sorted(set(all_words))

print(len(all_words))

70


In [22]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [23]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD

num_epochs = 1000
batch_size = 32
learning_rate = 0.003
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?
# TBD
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

In [24]:
# STEP 5: Save the model

# TBD: name and save the model
file = 'trained.pth'
torch.save(data, file)

In [26]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TBD: Load the intents file

data = torch.load('trained.pth')

# TBD: Retrieve the model and all the sizings

model_state = data['model_state']
input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']

# TBD: build the NN

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)
bot_name = 'Based-Bot'
print("Let's chat! \n If you'd like to finish chatting, type 'exit'.")

while True:
    sentence = input('You :')
    if sentence == 'exit':
        print('Thanks for chatting!')
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(prob)
    if prob.item()  > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand, try to be less cringe.")

Let's chat! 
 If you'd like to finish chatting, type 'exit'.
tensor(0.9988, device='cuda:0', grad_fn=<SelectBackward0>)
Based-Bot: Hello, thanks for visiting
tensor(0.9991, device='cuda:0', grad_fn=<SelectBackward0>)
Based-Bot: We sell pizza, hamburgers and tacos
tensor(0.4711, device='cuda:0', grad_fn=<SelectBackward0>)
Based-Bot: I do not understand, try to be less cringe.
Thanks for chatting!
